# Canonical System Identity and Analytical Grain

This notebook establishes the only admissible system-level representation
of residential solar installations used in downstream analysis.

It performs no exploratory analysis and introduces no analytical interpretation.

Its sole responsibilities are to:
- load raw Tracking the Sun data as reported,
- enforce identifier admissibility rules defined in Repo 1,
- formally declare the canonical system identifier,
- collapse multiple reported rows into one system-level record,
- enforce the one-row-per-system grain invariant,
- and produce auditable diagnostics for exclusions and violations.

No system-level artifact may be produced unless all identity and grain
constraints are satisfied.


## Phase 0 — Raw Data Load & Structural Inspection

This phase loads the raw Tracking the Sun dataset and performs basic structural inspection.
No assumptions are made about system identity, grain, or column roles.

The purpose of this phase is purely descriptive: to understand the shape, size, and
surface characteristics of the raw data before any admissibility or scope decisions
are applied.


In [2]:
from pathlib import Path
import os

import pandas as pd
import numpy as np

In [3]:
# Resolve raw data path via environment configuration

RAW_DATA_PATH = Path(os.environ.get("TRACKING_THE_SUN_DATA", ""))

if RAW_DATA_PATH == Path(""):
    raise EnvironmentError(
        "TRACKING_THE_SUN_DATA environment variable is not set."
    )

if not RAW_DATA_PATH.exists():
    raise FileNotFoundError(
        f"Raw Tracking the Sun dataset not found at: {RAW_DATA_PATH}"
    )

if RAW_DATA_PATH.suffix != ".parquet":
    raise ValueError(
        "TRACKING_THE_SUN_DATA must point to a .parquet file."
    )


In [4]:
# Load raw dataset
df_raw = pd.read_parquet(RAW_DATA_PATH)

In [5]:
df_raw.shape

(1921220, 80)

The raw Tracking the Sun dataset contains 1,921,220 rows and 80 columns.
Each row represents an administrative record related to a solar installation.
At this stage, rows are not assumed to correspond one-to-one with systems.

In [6]:
# Inspect column names
df_raw.columns.tolist()


['data_provider_1',
 'data_provider_2',
 'system_id_1',
 'system_id_2',
 'installation_date',
 'pv_system_size_dc',
 'total_installed_price',
 'rebate_or_grant',
 'customer_segment',
 'expansion_system',
 'multiple_phase_system',
 'tts_link_id',
 'new_construction',
 'tracking',
 'ground_mounted',
 'zip_code',
 'city',
 'utility_service_territory',
 'third_party_owned',
 'installer_name',
 'self_installed',
 'azimuth_1',
 'azimuth_2',
 'azimuth_3',
 'tilt_1',
 'tilt_2',
 'tilt_3',
 'module_manufacturer_1',
 'module_model_1',
 'module_quantity_1',
 'module_manufacturer_2',
 'module_model_2',
 'module_quantity_2',
 'module_manufacturer_3',
 'module_model_3',
 'module_quantity_3',
 'additional_modules',
 'technology_module_1',
 'technology_module_2',
 'technology_module_3',
 'bipv_module_1',
 'bipv_module_2',
 'bipv_module_3',
 'bifacial_module_1',
 'bifacial_module_2',
 'bifacial_module_3',
 'nameplate_capacity_module_1',
 'nameplate_capacity_module_2',
 'nameplate_capacity_module_3',
 '

The dataset contains 80 columns spanning identifiers, temporal fields,
physical system measurements, categorical indicators, and repeated
component specifications (modules, inverters, batteries).

At this stage, columns are not assigned semantic roles and no assumptions
are made about how values should be reconciled or interpreted.


In [7]:
# Inspect column data types
df_raw.dtypes

data_provider_1                object
data_provider_2                object
system_id_1                    object
system_id_2                    object
installation_date              object
                               ...   
battery_rated_capacity_kw     float64
battery_rated_capacity_kwh    float64
battery_price                 float64
technology_type                object
extensions_multiphase_id       object
Length: 80, dtype: object

Column data types reflect a typical administrative dataset structure.
Identifiers, categorical indicators, and dates are stored as objects,
while physical measurements and prices are numeric.

At this stage, no parsing, coercion, or semantic interpretation is applied.

In [8]:
# Missingness summary (top-level)
df_raw.isna().sum().sort_values(ascending=False).head(20)

micro_inverter_1               1830683
built_in_meter_inverter_1      1830683
bifacial_module_1              1778908
bipv_module_1                  1776393
output_capacity_inverter_1     1317603
micro_inverter_2                129121
built_in_meter_inverter_2       129121
bifacial_module_2                85966
bipv_module_2                    85878
output_capacity_inverter_2       78792
micro_inverter_3                 14075
built_in_meter_inverter_3        14075
bifacial_module_3                10889
bipv_module_3                    10883
output_capacity_inverter_3        8629
nameplate_capacity_module_1       1769
installation_date                  231
nameplate_capacity_module_2         55
nameplate_capacity_module_3         15
dc_optimizer                        12
dtype: int64

High missingness is concentrated in repeated component fields
(e.g. module_1–3, inverter_1–3, optional flags), reflecting structural
optionality rather than data loss. Core administrative and measurement
fields exhibit substantially lower missingness.

Phase 0 concludes here. No assertions about system identity, scope, or
column roles have been made.


## Phase 1 — System Identity Existence & Scope Resolution

This phase determines whether canonical system identity can be constructed
from the raw dataset and enforces scope restrictions required for
system-level analysis.

All assertions in this phase are gatekeeping conditions. Failure to meet
them halts or restricts the pipeline.


In [9]:
# Assert presence of canonical system identifier
if "tts_link_id" not in df_raw.columns:
    raise KeyError(
        "Canonical system identifier `tts_link_id` is missing from the dataset."
    )

df_raw["tts_link_id"].isna().sum()


0

### Canonical Identifier Completeness

The canonical system identifier (`tts_link_id`) is present for all rows in the dataset.
No null or missing values are observed.

This confirms that system identity is syntactically complete at the column level.
No scope restriction or pipeline halt is required due to missing identifier values.

### Identifier Scope Inspection

Although the canonical system identifier is present for all rows, not all
identifier values necessarily correspond to admissible real-world systems.

This step inspects the distribution of `tts_link_id` values to identify
sentinel or non-admissible identifiers that must be handled through explicit
scope rules before system-level analysis can proceed.


In [10]:
# Inspect distribution of canonical system identifier values
df_raw["tts_link_id"].value_counts().head(10)

tts_link_id
-1                         1656084
tts_extension_id_1           13872
tts_extension_id_41028         131
tts_extension_id_69328          12
tts_extension_id_146739         10
tts_extension_id_140488          9
tts_extension_id_53134           7
tts_extension_id_72722           6
tts_extension_id_121711          6
tts_extension_id_132802          6
Name: count, dtype: int64

### Interpretation: Identifier Scope and Admissibility

Inspection of the `tts_link_id` value distribution shows that the majority of
rows carry the identifier value `"-1"`. This value appears in approximately
1.66 million rows, far exceeding any other identifier.

The value `"-1"` functions as a sentinel indicating the absence of a valid
linked system identifier. Rows carrying this value do not assert membership
in an identifiable real-world solar installation and therefore cannot
participate in system-level analysis.

Treating `"-1"` as a valid identifier would collapse all unidentified records
into a single fictitious system, violating entity definition and grain
invariants. Accordingly, rows with `tts_link_id = "-1"` are classified as
out of scope for system-level processing.


In [11]:
# Enforce scope exclusion for non-admissible system identity
invalid_identity_mask = df_raw["tts_link_id"].astype(str).str.strip() == "-1"
excluded_count = invalid_identity_mask.sum()

df_raw = df_raw.loc[~invalid_identity_mask].copy()

print(
    f"Excluded {excluded_count:,} rows with non-admissible system identity "
    "(tts_link_id = '-1')."
)
print(f"Remaining rows: {len(df_raw):,}")

Excluded 1,656,084 rows with non-admissible system identity (tts_link_id = '-1').
Remaining rows: 265,136


### Phase 1 Conclusion — Identity Scope Resolved

Rows with non-admissible system identity (`tts_link_id = "-1"`) have been
explicitly excluded from the dataset. A total of 1,656,084 rows were removed
as out of scope for system-level analysis.

The remaining dataset contains 265,136 rows, each of which explicitly asserts
membership in an identifiable real-world system via a valid `tts_link_id`.

Canonical system identity is now fully defined, scoped, and admissible.


## Phase 2 — Identity Admissibility Diagnostics

With identity scope resolved, this phase evaluates whether the canonical
identifier (`tts_link_id`) behaves coherently as a system identifier.

Diagnostics in this phase assess multiplicity and reuse patterns without
modifying the dataset.


In [12]:
# Identity admissibility diagnostics
rows = len(df_raw)
unique_system_ids = df_raw["tts_link_id"].nunique()
rows_per_system_avg = rows / unique_system_ids

pd.DataFrame({
    "rows": [rows],
    "unique_system_ids": [unique_system_ids],
    "rows_per_system_avg": [rows_per_system_avg]
})


,rows,unique_system_ids,rows_per_system_avg
0,265136,123178,2.152462


### Interpretation: Identity Admissibility

After enforcing identity scope, the dataset contains 265,136 rows associated
with 123,178 unique system identifiers. This yields an average of approximately
2.15 rows per system.

This pattern confirms that `tts_link_id` behaves coherently as a canonical
system identifier: systems are reused across a small number of administrative
records rather than appearing as singletons or collapsing into a few dominant
identities.

The observed multiplicity is consistent with expected reporting behavior
(e.g. updates, expansions, or multi-phase records) and does not indicate
identity misuse or instability.


### Phase 2 Conclusion — Identity Admissibility Confirmed

Canonical system identity behaves consistently and plausibly across the
dataset. No identity anomalies are detected that would prevent system-level
collapse or grain enforcement.

The dataset is now ready for explicit system entity declaration and
row-to-system consolidation.


## Phase 3 — Canonical System Declaration

Based on prior admissibility checks, each distinct `tts_link_id` is treated
as representing a single real-world solar installation system.

The target analytical grain is defined as one row per system. All subsequent
transformations must preserve this invariant unless explicitly stated
otherwise.


**Entity:** Solar installation system  
**Canonical identifier:** `tts_link_id`  
**Target grain:** One row per system


## Phase 4 — Minimal Column-Role Binding for System Collapse

This phase binds concrete dataset columns to the semantic column-role
taxonomy defined in Repo 1 for the sole purpose of collapsing multiple
administrative rows into a single system-level record.

Bindings defined here are procedural and collapse-specific. They do not
introduce analytical interpretation, inference, or statistical treatment.
Full semantic use of column roles occurs in downstream notebooks.


Column-role bindings in this phase answer a narrow mechanical question:
how should multiple values for the same system be reconciled into one
record without violating epistemic constraints?

All bindings reference the column-role taxonomy in Repo 1 and are limited
to collapse mechanics (selection, aggregation, or exclusion).

For system-level collapse, columns are assigned one of the following
collapse behaviors:

1. **Group key**  
   Columns that define system identity and must be identical across rows.

2. **Deterministic select**  
   Columns where a single representative value is selected according to a
   fixed rule (e.g. latest, non-null).

3. **Aggregation**  
   Columns representing repeated components or additive quantities.

4. **Consistency check**  
   Columns that must agree across rows; disagreement is flagged.

5. **Excluded from system base**  
   Columns that are not admissible at the system grain and are dropped
   from the system-level artifact.


###  Within-System Variability Inspection

Before defining collapse rules, we inspect which columns vary across rows
belonging to the same system. Columns that are invariant within a system
can be safely collapsed; columns that vary require explicit reconciliation
rules or exclusion from the system-level base.


In [13]:
# Identify columns that vary within systems
variation_summary = (
    df_raw
    .groupby("tts_link_id")
    .agg(lambda x: x.nunique(dropna=True))
)

# Count how many systems have variation > 1 per column
columns_with_variation = (variation_summary > 1).sum().sort_values(ascending=False)

columns_with_variation.head(20)


multiple_phase_system          123168
system_id_1                    123168
installation_date              123165
expansion_system               122622
pv_system_size_dc              120184
inverter_loading_ratio         119640
total_installed_price          119307
azimuth_1                      106037
tilt_1                         101274
module_model_1                  93473
efficiency_module_1             92278
module_quantity_1               91607
nameplate_capacity_module_1     90638
inverter_model_1                82997
module_manufacturer_1           82502
installer_name                  78865
inverter_quantity_2             69014
module_manufacturer_2           65820
inverter_model_2                63626
inverter_manufacturer_2         62925
dtype: int64

In [14]:
# Columns that never vary within systems
invariant_columns = (variation_summary == 1).all()
invariant_columns[invariant_columns].index.tolist()


['extensions_multiphase_id']

### Minimal System-Level Schema Definition

Within-system variability analysis shows that nearly all raw columns vary
across administrative records for the same system. As a result, the
system-level base artifact produced in this notebook is intentionally
minimal.

The system-level schema captures:
- system identity,
- reporting multiplicity,
- and structural instability indicators.

No attempt is made to resolve physical system attributes at this stage.


In [15]:
# Construct minimal system-level diagnostics
system_base = (
    df_raw
    .groupby("tts_link_id")
    .agg(
        n_rows=("tts_link_id", "size"),
        n_installation_dates=("installation_date", "nunique"),
        n_system_sizes=("pv_system_size_dc", "nunique"),
        n_prices=("total_installed_price", "nunique"),
        has_expansion=("expansion_system", lambda x: x.fillna(False).any()),
        has_multiple_phases=("multiple_phase_system", lambda x: x.fillna(False).any()),
    )
    .reset_index()
)

system_base.head()


,tts_link_id,n_rows,n_installation_dates,n_system_sizes,n_prices,has_expansion,has_multiple_phases
0,tts_extension_id_1,13872,3594,4315,7166,True,True
1,tts_extension_id_10,2,2,2,2,True,True
2,tts_extension_id_100,2,2,2,2,True,True
3,tts_extension_id_1000,2,2,1,2,True,True
4,tts_extension_id_10000,2,2,2,2,True,True


### Phase 4 Conclusion — System-Level Base Constructed

Due to pervasive within-system variability across raw reporting records,
a minimal system-level base was constructed that preserves system identity
and records diagnostic indicators of reporting multiplicity and instability.

No physical system attributes were reconciled or inferred at this stage.
The resulting artifact serves as an index and diagnostic foundation for
downstream analytical notebooks.


In [16]:
# Write system-level base artifact
OUTPUT_PATH = Path("../outputs/system_level_base.parquet")
OUTPUT_PATH.parent.mkdir(parents=True, exist_ok=True)

system_base.to_parquet(OUTPUT_PATH, index=False)

OUTPUT_PATH


WindowsPath('../outputs/system_level_base.parquet')